# Redes de deslocamento para acesso ao parto hospitalar no Brasil

https://pcdas.icict.fiocruz.br/rede-de-parcerias/a-distancia-importa-redes-de-deslocamento-para-acesso-ao-parto-hospitalar-no-brasil/

## Importação de Bibliotecas

In [1]:
import numpy as np
import pandas as pd

In [2]:
from tqdm.notebook import tqdm
from gdown import download
from os.path import isfile
from zipfile import ZipFile
from pandas_profiling import ProfileReport

## Acesso aos Dados

In [3]:
URL = 'https://drive.google.com/uc?id=149l5K6rEXWZ6MhIfSsatSN_Q5Esum7x1'
DATASET_OUTPUT = 'data/SIHSUS.zip'

In [4]:
if isfile(DATASET_OUTPUT):
    zf = ZipFile(DATASET_OUTPUT)
else:
    download(URL, DATASET_OUTPUT, quiet=False)

## Dicionário de Variáveis

https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacoes-hospitalares-do-sus-sihsus/dicionario-de-variaveis/

In [5]:
COLUMNS = {
  'ANO_CMPT',
  'MUNIC_RES',
  'NASC',
  'PROC_REA', # ['0310010039', '0411010034']
  'DT_INTER',
  'MUNIC_MOV',
  'IDADE',
  'INSTRU',
  'CNES',
  'RACA_COR',
  'CGC_HOSP',
  'ano_internacao',
  'int_MSAUDCOD', 'int_RSAUDCOD', 'int_LATITUDE', 'int_LONGITUDE', 'int_REGIAO', 'int_SIGLA_UF', 'int_NOME_UF', 'int_MUNNOMEX',
  'res_SIGLA_UF', 'res_MUNNOMEX', 'res_MSAUDCOD', 'res_RSAUDCOD', 'res_LATITUDE', 'res_LONGITUDE', 'res_REGIAO', 'res_uf_CODIGO_UF', 'res_MUNCOD',
  'def_idade_anos', 'def_procedimento_realizado', 'def_rsaud_res', 'def_rsaud_int', 'def_regiao_int', 'def_regiao_res', 'def_uf_int', 'def_uf_sigla_int', 'def_uf_res', 'def_uf_sigla_res', 'def_municipio_int', 'def_municipio_res',
  'cod_uf_res', 'cod_uf_mov',
  'idade_nascimento_anos',
}

PROC_REA_PARTOS = {
  '0310010039',
  '0411010034',
}

DICT_OUTPUT = 'data/info/dict_cols.csv'

In [6]:
if isfile(DICT_OUTPUT):
    dict_cols = pd.read_csv(DICT_OUTPUT)
else:
    dict_sih = pd.read_csv(zf.open('dict_SIH.csv'), sep = ';')
    dict_sih['Tipo'] = dict_sih['Tipo'].map({
        'text' : 'object',
        'int8' : 'int64',
        'float8' : 'float32',
        'date' : 'object', # columns = {dt_inter, dt_saida}
    })
    dict_cols = dict_sih[dict_sih['Coluna'].isin(COLUMNS)]
    dict_cols.to_csv(DICT_OUTPUT)
dict_cols

,Unnamed: 0,Coluna,Tipo,Descrição
0,1,ANO_CMPT,int64,Ano de processamento da AIH
1,4,CGC_HOSP,object,CNPJ do estabelecimento
2,11,MUNIC_RES,int64,Município de residência do paciente
3,12,cod_uf_res,int64,Código da UF de residência do paciente
4,13,NASC,int64,Data de nascimento do paciente
5,31,PROC_REA,object,Procedimento realizado
6,46,DT_INTER,int64,Data de internação
7,61,MUNIC_MOV,int64,Município do estabelecimento
8,62,cod_uf_mov,int64,Código da UF do estabelecimento
9,65,IDADE,int64,Idade


## Tabelas Existentes

In [7]:
def get_row_from(fname):
  f = fname.split('.')[1].split('_')
  return {
    'File Name' : fname,
    'State' : f[1],
    'Year' : int(f[2]),
    'Month' : int(f[3]),
  }

In [8]:
tabelas = pd.DataFrame([
  get_row_from(fname)
    for fname in zf.namelist() if (
      fname[0] == 'E')
]).sort_values(
  by = ['State', 'Year', 'Month'],
  ignore_index = True)
tabelas

,File Name,State,Year,Month
0,ETLSIH.ST_AC_2008_1_t.csv,AC,2008,1
1,ETLSIH.ST_AC_2008_2_t.csv,AC,2008,2
2,ETLSIH.ST_AC_2008_3_t.csv,AC,2008,3
3,ETLSIH.ST_AC_2008_4_t.csv,AC,2008,4
4,ETLSIH.ST_AC_2008_5_t.csv,AC,2008,5
...,...,...,...,...
4395,ETLSIH.ST_TO_2021_3_t.csv,TO,2021,3
4396,ETLSIH.ST_TO_2021_4_t.csv,TO,2021,4
4397,ETLSIH.ST_TO_2021_5_t.csv,TO,2021,5
4398,ETLSIH.ST_TO_2021_6_t.csv,TO,2021,6


### Estados por ano que não possui os 12 meses completos

In [9]:
estados_ano = tabelas.groupby(['State', 'Year']).count()
estados_ano[estados_ano['Month'] != 12][['Month']].rename(columns={'Month': 'Months'})

Months
State Year        
AC    2009      11
      2021       7
AL    2021       7
AM    2021       7
AP    2021       7
BA    2021       7
CE    2021       7
DF    2021       7
ES    2021       7
GO    2021       7
MA    2021       7
MG    2021       7
MS    2021       7
MT    2021       7
PA    2021       7
PB    2021       7
PE    2021       7
PI    2021       7
PR    2021       7
RJ    2021       7
RN    2021       7
RO    2021       7
RR    2021       7
RS    2021       7
SC    2021       7
SE    2021       7
SP    2021       7
TO    2021       7

### Tabelas entre 2010 e 2020

- 27 Unidades Federativas;
- 11 Anos de cobertura;
- 12 Meses cada ano;
- 27 * 11 * 12 = 3564 tabelas.

In [10]:
tabelas = tabelas[(tabelas['Year'] >= 2010) & (tabelas['Year'] <= 2020)]
tabelas

,File Name,State,Year,Month
23,ETLSIH.ST_AC_2010_1_t.csv,AC,2010,1
24,ETLSIH.ST_AC_2010_2_t.csv,AC,2010,2
25,ETLSIH.ST_AC_2010_3_t.csv,AC,2010,3
26,ETLSIH.ST_AC_2010_4_t.csv,AC,2010,4
27,ETLSIH.ST_AC_2010_5_t.csv,AC,2010,5
...,...,...,...,...
4388,ETLSIH.ST_TO_2020_8_t.csv,TO,2020,8
4389,ETLSIH.ST_TO_2020_9_t.csv,TO,2020,9
4390,ETLSIH.ST_TO_2020_10_t.csv,TO,2020,10
4391,ETLSIH.ST_TO_2020_11_t.csv,TO,2020,11
